In [1]:
import os
import sys
import shutil
from pathlib import Path
import argparse
import subprocess
from pathlib import Path
from concurrent.futures import ThreadPoolExecutor, as_completed

In [2]:
%cd VideoGen/

/teamspace/studios/this_studio/VideoGen


In [3]:
!pwd

/teamspace/studios/this_studio/VideoGen


In [ ]:
!ffmpeg -i "data/03_old_cartoons_cropped/Whys_And_Other_Whys_(1927)_cropped.mp4" -vf "\
format=gray,\
eq=contrast=1.6:brightness=-0.08,\
unsharp=5:5:2.5" -c:a copy "data/03_old_cartoons_cropped/Whys_And_Other_Whys_(1927)_cropped_contrast_adj.mp4"

In [ ]:
!ffmpeg \
    -i \
    "data/03_old_cartoons_cropped/Hold_Anything_(1930)_cropped.mp4" \
    -vf \
    "cas=strength=0.5" \
    -c:a \
    copy \
    "data/03_old_cartoons_cropped/Hold_Anything_(1930)_cropped_enhanced.mp4"

In [ ]:
!ffmpeg -ss <middle_point> -i "data/02_old_cartoons_mp4/Lets_Sing_With_Popeye_(1934).mp4" -vf cropdetect -f null -t <analyze_duration> -

In [4]:
def organize_mp4_files(source_dir):
    # Create mp4_files directory if it doesn't exist
    target_dir = Path(source_dir) / "data/02_old_cartoons_mp4"
    
    # Get all files in the source directory
    files = os.listdir(f"{source_dir}/data/01_old_cartoons_raw")
    
    # Get all mp4 files
    mp4_files = [f for f in files if f.endswith('.mp4')]
    
    for mp4_file in mp4_files:
        source_path = Path(f"{source_dir}/data/01_old_cartoons_raw") / mp4_file
        base_name = mp4_file.removesuffix('.mp4')  # Remove .mp4 extension
        
        # Check for files with same name but different extensions
        similar_files = [
            f for f in files 
            if f.startswith(base_name + '.') 
            and f != mp4_file
        ]
        
        # If there are similar files or it's a single mp4, process it
        if similar_files or len([f for f in files if f.startswith(base_name + '.')]) == 1:
            target_path = target_dir / mp4_file
            shutil.copy2(source_path, target_path)
            print(f"Processed: {mp4_file}")

In [ ]:
current_dir = os.getcwd()
organize_mp4_files(current_dir)

In [4]:
def convert_to_mp4(input_file, output_dir):
    """Convert a video file to MP4 format."""
    try:
        command = [
            'ffmpeg',
            '-i', str(input_file),
            '-c:v', 'libx264',
            '-c:a', 'aac',
            '-y',
            str(output_dir)
        ]
        
        process = subprocess.run(
            command,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True
        )
        return process.returncode == 0
    except Exception as e:
        print(f"Error converting {input_file}: {str(e)}")
        return False

In [5]:
def detect_black_bars(input_file, analyze_duration=10):
    # First get video duration
    duration_cmd = [
        'ffprobe',
        '-v', 'quiet',
        '-show_entries', 'format=duration',
        '-of', 'default=noprint_wrappers=1:nokey=1',
        str(input_file)
    ]
    duration = float(subprocess.check_output(duration_cmd).decode().strip())
    
    # Calculate middle point
    middle_point = max(0, (duration / 2) - (analyze_duration / 2))
    
    # Modified detection command to start from middle
    detect_command = [
        'ffmpeg',
        '-ss', str(middle_point),  # Seek to middle of video
        '-i', str(input_file),
        '-vf', 'cropdetect',
        '-f', 'null',
        '-t', str(analyze_duration),  # Analyze for specified duration
        '-'
    ]

    process = subprocess.run(
        detect_command,
        stderr=subprocess.PIPE,
        text=True
    )

    crop_lines = [line for line in process.stderr.split('\n') if 'crop=' in line]
    crop_params = crop_lines[-1].split('crop=')[1].split(' ')[0]
    
    return crop_params

In [6]:
def remove_black_bars(input_path, output_path, crop_params):
    """Remove black bars using detected crop parameters"""
    cmd = [
        'ffmpeg',
        '-i', str(input_path),
        '-vf', f'crop={crop_params}',
        '-c:a', 'copy',
        str(output_path)
    ]
    try:
        subprocess.run(cmd, check=True, capture_output=True)
    except subprocess.CalledProcessError as e:
        logging.error(f"Cropping failed for {input_path}: {e}")

In [10]:
def process_video(input_file):
    """Process a single video file. Returns (input_file, output_file, task_type, success)."""
    try:
        print(f"Processing: {input_file.name}")
        
        # Convert to MP4 if needed
        if input_file.suffix.lower() != '.mp4':
            output_file_mp4 = f"{os.getcwd()}/data/02_old_cartoons_mp4/{input_file.name.split('.')[0]}.mp4"
            convert_to_mp4(input_file, output_file_mp4)

        # Remove black bars
        # output_file_cropped = f"{os.getcwd()}/data/03_old_cartoons_cropped/{input_file.name.split('.')[0]}_cropped.mp4"
        # crop_params = detect_black_bars(input_file=input_file)
        # if crop_params:
        #     remove_black_bars(input_file, output_file_cropped, crop_params)
        # else:
        #     shutil.copy(input_file.name, output_file_cropped)

    except Exception as e:
        print(f"Error processing {input_file}: {str(e)}")

In [11]:
process_video(Path("data/01_old_cartoons_raw/The_Mad_Doctor_(1933).webm"))

Processing: The_Mad_Doctor_(1933).webm


In [102]:
dir_path = Path(f"{os.getcwd()}/data/02_old_cartoons_mp4")
if not dir_path.is_dir():
    print(f"Error: Directory '{dir_path}' does not exist")
    sys.exit(1)

# Prepare tasks
# video_extensions = {'.ogv', '.mkv', '.mov', '.wmv', '.flv', '.webm'}
video_extensions = {".mp4"}
tasks = []

# Add conversion tasks
for file_path in dir_path.iterdir():
    if file_path.suffix.lower() in video_extensions:
        tasks.append({'input_file': file_path})

In [103]:
print(tasks)

[{'input_file': PosixPath('/teamspace/studios/this_studio/VideoGen/data/02_old_cartoons_mp4/A_Date_To_Skate_(1938).mp4')}, {'input_file': PosixPath('/teamspace/studios/this_studio/VideoGen/data/02_old_cartoons_mp4/Africa_Before_Dark_(1928).mp4')}, {'input_file': PosixPath('/teamspace/studios/this_studio/VideoGen/data/02_old_cartoons_mp4/Alice_Gets_Stung_(1925).mp4')}, {'input_file': PosixPath('/teamspace/studios/this_studio/VideoGen/data/02_old_cartoons_mp4/Alice_Solves_the_Puzzle_(1925).mp4')}, {'input_file': PosixPath('/teamspace/studios/this_studio/VideoGen/data/02_old_cartoons_mp4/Alice_the_Toreador_(1925).mp4')}, {'input_file': PosixPath('/teamspace/studios/this_studio/VideoGen/data/02_old_cartoons_mp4/Alices_Orphan_(1926).mp4')}, {'input_file': PosixPath('/teamspace/studios/this_studio/VideoGen/data/02_old_cartoons_mp4/All_Wet_(1927).mp4')}, {'input_file': PosixPath('/teamspace/studios/this_studio/VideoGen/data/02_old_cartoons_mp4/Battling_Bosko_(1931).mp4')}, {'input_file': Posi

In [ ]:
with ThreadPoolExecutor(max_workers=8) as executor:
    # Submit all tasks and get futures
    futures = [
        executor.submit(process_video, task['input_file'])
        for task in tasks
    ]
    
    # Process completed tasks as they finish
    for future in as_completed(futures):
        future.result()